# Imports

In [ ]:
import mdtraj as md
import numpy as np
from datetime import datetime
from tqdm import tqdm
from pathlib import Path
import yaml
import plotly.express as px

# Analyze Split PDBs

In [ ]:
idir = Path("/Users/alexpayne/lilac-mount-point/asap-datasets-ap/sars_01_prepped_v3_split/")

In [ ]:
pdbs = list(idir.glob("*active_site.pdb"))

In [ ]:
len(pdbs)

In [ ]:
ts = [md.load(pdb) for pdb in tqdm(pdbs)]

In [ ]:
ids = np.array(list([pdbs[i] for i, t in enumerate(ts)]))

In [ ]:
lens = np.array(list([t.n_atoms for i, t in enumerate(ts)]))

In [ ]:
ids[lens == 354]

In [ ]:
sum(lens == 354)

In [ ]:
ids[lens != 354]

In [ ]:
same_atoms = ids[lens == 354]

In [ ]:
combined = md.load(list(same_atoms))

In [ ]:
combined.save_pdb("527_combined.pdb")

In [ ]:
combined = md.load("527_combined.pdb")

## calculate RMSD

In [ ]:
rmsd_array = []
for i in range(combined.n_frames):
    rmsd_array.append(md.rmsd(combined, combined, frame=i))
rmsd_matrix = np.matrix(rmsd_array)

In [ ]:
np.save("527_rmsd_array.npy", rmsd_matrix)

In [ ]:
np.shape(rmsd_array)

In [ ]:
np.shape(rmsd_matrix)

In [ ]:
rmsd_matrix * 10

In [ ]:
rmsd_matrix.max()

In [ ]:
rmsd_matrix.min()

In [ ]:
np.shape(rmsd_matrix)

## is matrix symmetric?

In [ ]:
np.all(rmsd_matrix - rmsd_matrix.T) < 0.00001

### Yes!

## Plot RMSD Matrix

In [ ]:
rmsds = np.triu(rmsd_matrix).flatten()
rmsds = rmsds[rmsds.nonzero()]

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(rmsds)

## RMSD Matrix to dataframe

In [ ]:
import pandas as pd
df = pd.DataFrame(rmsd_matrix)

In [ ]:
# <https://stackoverflow.com/questions/60082349/pivot-table-to-tidy-data-frame-in-pandas>
tidydf = df.rename_axis(index='V1',columns="V2").reset_index().melt("V1")

In [ ]:
tidydf.columns = ['i','j','RMSD']

### plot using plotly

In [ ]:
fig = px.scatter(tidydf, x='i', y='j', color='RMSD')

In [ ]:
fig.write_image("test.png")

# Compute Backbone Dihedrals

In [ ]:
phi = md.compute_phi(combined)

In [ ]:
np.shape(phi[1])

In [ ]:
phi[1]

# Cluster

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
rmsd_matrix = np.load("527_rmsd_array.npy")

## calculate PCA

In [ ]:
pca = PCA(n_components='mle')

In [ ]:
pca.fit(rmsd_matrix)

In [ ]:
pca.n_components

In [ ]:
pca.explained_variance_

## plot with matplotlib

In [ ]:
import matplotlib.pyplot as plt
cum_sum_exp = np.cumsum(pca.explained_variance_)
plt.bar(range(0,len(pca.explained_variance_)), pca.explained_variance_, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_exp)), cum_sum_exp, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
pca.n_components_

In [ ]:
transformed = pca.transform(rmsd_matrix)

In [ ]:
np.shape(transformed)

In [ ]:
np.shape(transformed[:,0:2])

In [ ]:
plt.scatter(transformed[:,0], transformed[:,1])

In [ ]:
plt.scatter(transformed[:,0], transformed[:,2])

In [ ]:
plt.scatter(transformed[:,1], transformed[:,2])

## kmeans cluster

In [ ]:
plt.scatter(transformed[:,2], transformed[:,0])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=10)

In [ ]:
kmeans.fit(rmsd_matrix)

In [ ]:
kmeans.labels_

## plot with labels

In [ ]:
plt.scatter(transformed[:,0], transformed[:,1], c=kmeans.labels_)

In [ ]:
plt.scatter(transformed[:,0], transformed[:,2], c=kmeans.labels_)

In [ ]:
plt.scatter(transformed[:,1], transformed[:,2], c=kmeans.labels_)

In [ ]:
x = 5
y = 5
fig, axs = plt.subplots(x,y)
for i in range(x):
    for j in range(y):
        ax = axs[i,j]
        ax.scatter(transformed[:,i], transformed[:,j], c=kmeans.labels_)

In [ ]:
fig

## convert to pandas array

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(transformed[:, 0:5])

In [ ]:
df["Cluster"] = kmeans.labels_

In [ ]:
df

In [ ]:
pd.plotting.scatter_matrix(df)

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter_matrix(df, dimensions=[0,1,2,3,4], color='Cluster')

In [ ]:
fig.show()